In [15]:
import psycopg2
import sys
import pandas as pd
import argparse

In [16]:
parser = argparse.ArgumentParser()
parser.add_argument('--file_path', type=str)
parser.add_argument('--sch_name', type=str)
parser.add_argument('--tb_name', type=str)
parser.add_argument('--user', type=str)
parser.add_argument('--password', type=str)

_StoreAction(option_strings=['--password'], dest='password', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [17]:
# Creates schema
def create_schema(sch_name, tb_name, cursor, conn):
    query="CREATE SCHEMA IF NOT EXISTS {0}; DROP TABLE IF EXISTS {0}.{1};".format(sch_name, tb_name)
    cursor.execute(query)
    conn.commit()


In [25]:
# Creates table based on csv
def create_table(sch_name, tb_name, data, cursor, conn):
    query = "CREATE TABLE IF NOT EXISTS {}.{} ( pk VARCHAR PRIMARY KEY, ".format(sch_name, tb_name)
    data_columns = [col for col in data.columns if col != 'pk']
    for i, col in enumerate(data_columns):
        # Columns
        col_type=data.dtypes[i+1]
        if col_type == 'int64':
            postgres_type = 'INT'
        elif col_type == 'float64':
            postgres_type = 'NUMERIC'
        else:
            postgres_type = 'VARCHAR'
        string_var = "{} {} NULL".format(col, postgres_type)
        
        if i==0:
            query = query + string_var
        else:
            query = query + ", " + string_var
    query = query + ")"
    
    cursor.execute(query)
    conn.commit()

In [19]:
# Upload data
def upload_data(sch_name, tb_name, file_path, cursor, conn):
    table = "{}.{}".format(sch_name, tb_name)
    f = open(file_path, 'r')
    next(f) # skips header
    cursor.copy_from(f, table, sep=',', null='None')
    f.close()
    conn.commit()

In [20]:
def main(user, password, file_path, sch_name, tb_name):
    conn_string = "host='localhost' dbname='postgres' user='{}' password='{}'".format(user, password)
    print("Connecting to database\n     {0}".format(conn_string))

    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print("Connected!\n")

    # Reads data from file
    print("Reading data...")
    data = pd.read_csv(file_path)

    # Create/check schrma
    print("Creating schema...")
    create_schema(sch_name=sch_name, tb_name=tb_name, cursor=cursor, conn=conn)

    # Create data
    print("Creating table...")
    create_table(sch_name=sch_name, tb_name=tb_name, data=data, cursor=cursor, conn=conn)

    # Upload data
    print("Uploading data...")
    upload_data(sch_name=sch_name, tb_name=tb_name, file_path=file_path, cursor=cursor, conn=conn)

In [26]:
#args = parser.parse_args()
main(user='postgres', password='dummy', sch_name = 'census_test', tb_name='census_oh', file_path = 'data.csv')

Connecting to database
     host='localhost' dbname='postgres' user='postgres' password='dummy'
Connected!

Reading data...
Creating schema...
Creating table...
Uploading data...


In [28]:
# CHECK
conn_string = "host='l{ocalhost' dbname='postgres' user='{}' password='{}'".format(user, password)
print("Connecting to database\n     {0}".format(conn_string))

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()}
cursor.execute("SELECT * from census_test.census_oh;")
cursor.fetchall()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block
